# Model Experimentation
## Masters in AI & ML Project

This notebook experiments with different ML models and hyperparameter tuning.

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.data_processing import DataProcessor
from src.model_development import ModelDeveloper
from src.model_evaluation import ModelEvaluator
from src.utils import save_model, set_plot_style
import warnings
warnings.filterwarnings('ignore')

set_plot_style()
print('✓ Libraries imported successfully')

## 1. Load and Prepare Data

In [ ]:
# Load dataset using DataProcessor
# For demonstration, we'll create a sample dataset
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15,
                          n_redundant=5, n_classes=2, random_state=42)
df = pd.DataFrame(X, columns=[f'Feature_{i+1}' for i in range(20)])
df['Target'] = y

# Save to CSV for DataProcessor
temp_path = '../data/raw/sample_data.csv'
df.to_csv(temp_path, index=False)

# Use DataProcessor
processor = DataProcessor(temp_path)
processor.load_data()
processor.explore_data()

In [ ]:
# Prepare data for modeling
X_train, X_test, y_train, y_test = processor.prepare_data(
    target_column='Target',
    test_size=0.2,
    scale=True
)

print(f'\n✓ Data prepared:')
print(f'   Training set: {X_train.shape}')
print(f'   Test set: {X_test.shape}')

## 2. Train Multiple Models

In [ ]:
# Initialize ModelDeveloper
developer = ModelDeveloper(task_type='classification', random_state=42)

# Train all default models
trained_models = developer.train_models(X_train, y_train)

## 3. Evaluate Models

In [ ]:
# Initialize ModelEvaluator
evaluator = ModelEvaluator(task_type='classification')

# Evaluate all models
comparison_df = evaluator.evaluate_models(trained_models, X_test, y_test)

# Display results
comparison_df

In [ ]:
# Visualize model comparison
fig = evaluator.plot_model_comparison(comparison_df, metric='Accuracy')
plt.show()

fig = evaluator.plot_model_comparison(comparison_df, metric='F1-Score')
plt.show()

## 4. Detailed Analysis of Best Models

In [ ]:
# Get top 3 models
top_models = comparison_df.head(3)['Model'].tolist()
print(f'\n🏆 Top 3 Models: {top_models}')

# Analyze each top model
for model_name in top_models:
    print(f'\n{"="*60}')
    print(f'Analyzing: {model_name}')
    print("="*60)
    
    model = trained_models[model_name]
    y_pred = model.predict(X_test)
    
    # Confusion Matrix
    fig = evaluator.plot_confusion_matrix(y_test, y_pred, model_name)
    plt.show()
    
    # ROC Curve (if probabilities available)
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test)
        fig = evaluator.plot_roc_curve(y_test, y_pred_proba, model_name)
        plt.show()
    
    # Feature Importance (if available)
    if hasattr(model, 'feature_importances_'):
        fig = evaluator.plot_feature_importance(
            model, X_train.columns.tolist(), model_name, top_n=15
        )
        plt.show()

## 5. Cross-Validation

In [ ]:
# Perform cross-validation on top models
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])

cv_results = {}
for model_name in top_models:
    model = trained_models[model_name]
    cv_result = developer.cross_validate_model(model, X_full, y_full, cv=5)
    cv_results[model_name] = cv_result
    print(f'{model_name}: {cv_result["mean"]:.4f} ± {cv_result["std"]:.4f}')

# Visualize CV results
cv_means = [cv_results[name]['mean'] for name in top_models]
cv_stds = [cv_results[name]['std'] for name in top_models]

plt.figure(figsize=(10, 6))
plt.barh(top_models, cv_means, xerr=cv_stds, capsize=5)
plt.xlabel('Cross-Validation Accuracy')
plt.title('5-Fold Cross-Validation Results')
plt.tight_layout()
plt.show()

## 6. Hyperparameter Tuning

In [ ]:
# Tune the best performing model
best_model_name = comparison_df.iloc[0]['Model']
print(f'\n🔧 Tuning hyperparameters for: {best_model_name}')

# Get parameter grid (example for Random Forest)
if 'Random Forest' in best_model_name:
    from src.model_development import CLASSIFICATION_PARAM_GRIDS
    param_grid = CLASSIFICATION_PARAM_GRIDS['Random Forest']
    
    from sklearn.ensemble import RandomForestClassifier
    base_model = RandomForestClassifier(random_state=42)
    
    # Perform tuning (this may take a while)
    tuned_model = developer.hyperparameter_tuning(
        base_model, param_grid, X_train, y_train, cv=3
    )
    
    # Evaluate tuned model
    tuned_metrics = evaluator.evaluate_model(
        tuned_model, X_test, y_test, f'{best_model_name} (Tuned)'
    )
    
    print(f'\n📊 Tuned Model Performance:')
    print(f'   Accuracy: {tuned_metrics["accuracy"]:.4f}')
    print(f'   F1-Score: {tuned_metrics["f1_score"]:.4f}')

## 7. Save Best Model

In [ ]:
# Save the best model
best_model = trained_models[best_model_name]
model_path = f'../models/best_model_{best_model_name.replace(" ", "_")}.joblib'

save_model(best_model, model_path)
print(f'\n✓ Best model saved: {best_model_name}')

## 8. Generate Final Report

In [ ]:
# Generate detailed report for best model
report = evaluator.generate_evaluation_report(best_model_name)
print(report)

# Save report to file
report_path = f'../results/model_report_{best_model_name.replace(" ", "_")}.txt'
with open(report_path, 'w') as f:
    f.write(report)
print(f'\n✓ Report saved to: {report_path}')

## 9. Summary

In [ ]:
print('\n' + '='*60)
print('MODEL EXPERIMENTATION SUMMARY')
print('='*60)
print(f'\n🏆 Best Model: {best_model_name}')
print(f'📊 Test Accuracy: {comparison_df.iloc[0]["Accuracy"]:.4f}')
print(f'📈 F1-Score: {comparison_df.iloc[0]["F1-Score"]:.4f}')
print(f'\n✓ Total models trained: {len(trained_models)}')
print(f'✓ Best model saved to: {model_path}')

print('\n' + '='*60)
print('RECOMMENDATIONS')
print('='*60)
print('1. Further hyperparameter tuning for improved performance')
print('2. Ensemble methods combining top models')
print('3. Feature engineering based on domain knowledge')
print('4. Collect more data if model performance is insufficient')
print('5. Consider model interpretability for deployment')
print('\n✓ Model Experimentation Complete!')